In [2]:
import pandas as pd
import csv
import re
import statsmodels.api as sm


from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [3]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/sc_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [4]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

sf_file = file_url.format('sf_airbnb/listings_details')
ok_file = file_url.format('ok_airbnb/listings_details')
sc_file = file_url.format('sc_airbnb/listings_details')
scz_file = file_url.format('scz_airbnb/listings_details')

In [5]:
date_to_parse = ['last_scraped','host_since', 'first_review', 'last_review']

listings_sf = pd.read_csv(sf_file, parse_dates=date_to_parse)
listings_ok = pd.read_csv(ok_file, parse_dates=date_to_parse)
listings_sc = pd.read_csv(sc_file, parse_dates=date_to_parse)
listings_scz = pd.read_csv(scz_file, parse_dates=date_to_parse)

In [6]:
print(listings_sf.shape, listings_ok.shape, listings_sc.shape, listings_scz.shape)

(6633, 96) (2898, 96) (5668, 96) (1570, 96)


In [7]:
# Created keys for future references

frames = [listings_sf, listings_ok, listings_sc, listings_scz]
keys = ['listings_sf', 'listings_ok', 'listings_sc', 'listings_scz']

In [8]:
df_listings = pd.concat(frames, keys=keys)
df_listings.head()

id                        listing_url     scrape_id  \
listings_sf 0   958   https://www.airbnb.com/rooms/958  2.018080e+13   
            1  5858  https://www.airbnb.com/rooms/5858  2.018080e+13   
            2  7918  https://www.airbnb.com/rooms/7918  2.018080e+13   
            3  8142  https://www.airbnb.com/rooms/8142  2.018080e+13   
            4  8339  https://www.airbnb.com/rooms/8339  2.018080e+13   

              last_scraped                                               name  \
listings_sf 0   2018-08-06                Bright, Modern Garden Unit - 1BR/1B   
            1   2018-08-06                                 Creative Sanctuary   
            2   2018-08-06         A Friendly Room - UCSF/USF - San Francisco   
            3   2018-08-06  Friendly Room Apt. Style -UCSF/USF - San Franc...   
            4   2018-08-06                    Historic Alamo Square Victorian   

                                                         summary  \
listings_sf 0  Our bright garden unit overlooks a grassy back...   
            1                                                NaN   
            2  Nice and good public transportation.  7 minute...   
            3  Nice and good public transportation.  7 minute...   
            4  Pls email before booking.  Interior featured i...   

                                                           space  \
listings_sf 0  Newly remodeled, modern, and bright garden uni...   
            1  We live in a large Victorian house on a quiet ...   
            2  Room rental-sunny view room/sink/Wi Fi (inner ...   
            3  Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...   
            4  Please send us a quick message before booking ...   

                                                     description  \
listings_sf 0  Our bright garden unit overlooks a grassy back...   
            1  We live in a large Victorian house on a quiet ...   
            2  Nice and good public transportation.  7 minute...   
            3  Nice and good public transportation.  7 minute...   
            4  Pls email before booking.  Interior featured i...   

              experiences_offered  \
listings_sf 0                none   
            1                none   
            2                none   
            3                none   
            4                none   

                                           neighborhood_overview  \
listings_sf 0  *Quiet cul de sac in friendly neighborhood *St...   
            1  I love how our neighborhood feels quiet but is...   
            2  Shopping old town, restaurants, McDonald, Whol...   
            3                                                NaN   
            4                                                NaN   

                     ...         requires_license      license  \
listings_sf 0        ...                        t  STR-0001256   
            1        ...                        t          NaN   
            2        ...                        t          NaN   
            3        ...                        t          NaN   
            4        ...                        t  STR-0000264   

              jurisdiction_names instant_bookable is_business_travel_ready  \
listings_sf 0  {"SAN FRANCISCO"}                f                        f   
            1  {"SAN FRANCISCO"}                f                        f   
            2  {"SAN FRANCISCO"}                f                        f   
            3  {"SAN FRANCISCO"}                f                        f   
            4  {"SAN FRANCISCO"}                f                        f   

                       cancellation_policy  require_guest_profile_picture  \
listings_sf 0                     moderate                              f   
            1  strict_14_with_grace_period                              f   
            2  strict_14_with_grace_period                              f   
            3  strict_14_with_grace_period                      

In [9]:
df_listings.reset_index()

,level_0,level_1,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,listings_sf,0,958,https://www.airbnb.com/rooms/958,2.018080e+13,2018-08-06,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,...,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,listings_sf,1,5858,https://www.airbnb.com/rooms/5858,2.018080e+13,2018-08-06,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,listings_sf,2,7918,https://www.airbnb.com/rooms/7918,2.018080e+13,2018-08-06,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16
3,listings_sf,3,8142,https://www.airbnb.com/rooms/8142,2.018080e+13,2018-08-06,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.15
4,listings_sf,4,8339,https://www.airbnb.com/rooms/8339,2.018080e+13,2018-08-06,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,...,t,STR-0000264,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,t,t,2,0.24
5,listings_sf,5,8567,https://www.airbnb.com/rooms/8567,2.018080e+13,2018-08-06,Lovely 2BR flat Great Location,NaN,"Fully furnished 2BR, 1BA flat in beautiful Vic...","Fully furnished 2BR, 1BA flat in beautiful Vic...",...,t,STR-0000278,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,0.27
6,listings_sf,6,8739,https://www.airbnb.com/rooms/8739,2.018080e+13,2018-08-06,"Mission Sunshine, with Private Bath","Welcome to San Francisco's Mission District, t...","Your sunny room has a queen size bed, and look...","Welcome to San Francisco's Mission District, t...",...,t,STR-0000028,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,2,5.41
7,listings_sf,7,9225,https://www.airbnb.com/rooms/9225,2.018080e+13,2018-08-06,Artful Potrero Separate Floor with Garden,A Unique Guest Suite! A Spacious Art Filled ...,"It is unique, very spacious [800 sq. ft. with...",A Unique Guest Suite! A Spacious Art Filled ...,...,t,STR-0000179,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,3.84
8,listings_sf,8,10251,https://www.airbnb.com/rooms/10251,2.018080e+13,2018-08-06,Victorian Suite in Inner Mission,NaN,Please read this before you book! Second floor...,Please read this before you book! Second floor...,...,t,STR-0001558,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,2.69
9,listings_sf,9,10578,https://www.airbnb.com/rooms/10578,2.018080e+13,2018-08-06,Classic Nob Hill Studio - Roof Deck,A cute studio with nice street views and lots ...,"Features: Elevator, Wood floors, TV, DVD Play...",A cute studio with nice street views and lots ...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,moderate,f,t,1,0.22


In [10]:
# Creat additional column to keep track of original district

df_listings['district'] = ''

df_listings.loc['listings_sf']['district'] = 'San Francisco'
df_listings.loc['listings_ok']['district'] = 'Oakland'
df_listings.loc['listings_sc']['district'] = 'Santa Clara'
df_listings.loc['listings_scz']['district'] = 'Santa Cruz'

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [11]:
df_district =  df_listings.loc[:, ['id', 'district']]
df_district.head()

id       district
listings_sf 0   958  San Francisco
            1  5858  San Francisco
            2  7918  San Francisco
            3  8142  San Francisco
            4  8339  San Francisco

In [11]:
# Check Null vales within data

df_listings.isnull().sum()

id                                      0
listing_url                             0
scrape_id                               0
last_scraped                            0
name                                    2
summary                               468
space                                3489
description                           124
experiences_offered                     0
neighborhood_overview                5398
notes                                7321
transit                              5550
access                               5261
interaction                          5880
house_rules                          4658
thumbnail_url                       16769
medium_url                          16769
picture_url                             0
xl_picture_url                      16769
host_id                                 0
host_url                                0
host_name                               0
host_since                              0
host_location                     

In [12]:
df_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [13]:
# Noted >90% null value in 'experience_offered', 'host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet'
# Remove 'jurisdiction_names'
# Removed 'neighbourhood',  kept'neighbourhood_cleansed',
# Both city and 'smart location' provides city level information, and 'smart_location can be dropped
# Country code and country can be dropped, as all data located within US
# City is parent of neighbourhood_cleansed

columns_to_keep =[
       'id','host_since','host_response_time', 'host_response_rate',
        'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 
       'host_has_profile_pic', 'host_identity_verified', 
        'neighbourhood_cleansed',
       'city', 
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type','amenities', 
       'price', 'security_deposit',
       'cleaning_fee', 'guests_included',  'minimum_nights','number_of_reviews',
       'first_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month', 'district']

In [14]:
df_smaller_listings = df_listings[columns_to_keep]

In [15]:
df_smaller_listings

id host_since  host_response_time host_response_rate  \
listings_sf  0          958 2008-07-31      within an hour                92%   
             1         5858 2009-03-02      within an hour               100%   
             2         7918 2009-06-17  within a few hours               100%   
             3         8142 2009-06-17  within a few hours               100%   
             4         8339 2009-07-02      within an hour               100%   
             5         8567 2009-07-14        within a day                80%   
             6         8739 2009-01-27      within an hour               100%   
             7         9225 2009-08-04      within an hour               100%   
             8        10251 2009-08-31      within an hour               100%   
             9        10578 2009-09-08      within an hour               100%   
             10       10819 2009-09-15        within a day                99%   
             11       10820 2009-09-15        within a day                99%   
             12       10824 2009-09-15        within a day                99%   
             13       10832 2009-09-15        within a day                99%   
             14       12041 2009-06-17  within a few hours               100%   
             15       12042 2009-06-17  within a few hours               100%   
             16       12522 2009-05-22      within an hour               100%   
             17       12584 2009-10-29  within a few hours               100%   
             18       14125 2009-11-18  within a few hours               100%   
             19       17132 2009-10-04      within an hour               100%   
             20       18231 2010-01-13  within a few hours               100%   
             21       18904 2010-01-21  within a few hours               100%   
             22       19040 2010-01-22  within a few hours               100%   
             23       21334 2010-02-15      within an hour               100%   
             24       21914 2009-06-17  within a few hours               100%   
             25       23540 2010-03-13      within an hour               100%   
             26       23611 2009-05-25      within an hour               100%   
             27       23630 2010-03-14      within an hour               100%   
             28       23680 2010-03-15  within a few hours               100%   
             29       24450 2010-03-25      within an hour               100%   
...                     ...        ...                 ...                ...   
listings_scz 1540  27581724 2018-06-29                 NaN                NaN   
             1541  27599937 2011-01-10      within an hour               100%   
             1542  27627707 2014-11-28                 NaN                NaN   
             1543  27663827 2014-02-25      within an hour               100%   
             1544  27697491 2018-08-12      within an hour                80%   
             1545  27700661 2017-10-21      within an hour               100%   
             1546  27706555 2015-12-26  within a few hours               100%   
             1547  27784211 2017-10-12      within an hour               100%   
             1548  27784627 2014-08-21      within an hour               100%   
             1549  27815674 2016-07-26      within an hour               100%   
             1550  27850750 2018-08-18      within an hour               100%   
             1551  27853574 2018-08-18      within an hour               100%   
             1552  27854924 2018-08-18      within an hour               100%   
             1553  27950686 2014-09-29                 NaN                NaN   
             1554  27975900 2017-11-10        within a day               100%   
             1555  27980223 2017-11-10        within a day               100%   
             1556  27983194 2014-08-02  within a few hours                54%   
             1557  28000915 2017-01-16      within an ho

In [16]:
# This is to onvert strings with percentage sign to float

def percent_to_float(string):
    """
    Convert strings with percentage sign to float
    """
    percentage = float(str(string).strip('%'))/100
    return percentage

In [17]:
# Convert host_response_rate to float

df_smaller_listings['host_response_rate'] = df_smaller_listings['host_response_rate']  \
                                            .apply(lambda x: percent_to_float(x))

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
def currency_to_float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [19]:
# Convert currency features to float

df_smaller_listings['price'] = df_smaller_listings['price'].apply(lambda x: currency_to_float(x))
df_smaller_listings['security_deposit'] = df_smaller_listings['security_deposit'].apply(lambda x: currency_to_float(x))
df_smaller_listings['cleaning_fee'] = df_smaller_listings['cleaning_fee'].apply(lambda x: currency_to_float(x))

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [20]:
df_smaller_listings.shape

(16769, 43)

In [21]:
df_smaller_listings.price.describe(percentiles=[.05, .10, .25, .5, .75, .90, .95])

count    16769.000000
mean       188.023555
std        274.564274
min          0.000000
5%          41.000000
10%         52.000000
25%         79.000000
50%        125.000000
75%        201.000000
90%        350.000000
95%        499.000000
max      10000.000000
Name: price, dtype: float64

In [22]:
# Remove outliers. Use 5% and 95% percentiles as cutoff

df_smaller_listings = df_smaller_listings[(df_smaller_listings['price']<=500) 
                                         & (df_smaller_listings['price']>=40)]

In [23]:
df_smaller_listings.shape

(15379, 43)

In [24]:
# Remove special characters within Amenities

df_smaller_listings['jurisdiction_names'] = df_smaller_listings['jurisdiction_names'].str.replace('[^\w\s,]', '')
df_smaller_listings['amenities'] = df_smaller_listings['amenities'].str.replace('[^\w\s,]', '')

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [25]:
df_smaller_listings.amenities.head()

listings_sf  0    TV,Cable TV,Internet,Wifi,Kitchen,Pets live on...
             1    Internet,Wifi,Kitchen,Heating,Familykid friend...
             2    TV,Internet,Wifi,Kitchen,Free street parking,H...
             3    TV,Internet,Wifi,Kitchen,Free street parking,H...
             5    TV,Cable TV,Internet,Wifi,Kitchen,Free parking...
Name: amenities, dtype: object

In [26]:
df_amenities = df_smaller_listings  \
               .amenities  \
               .str.get_dummies(sep = ",")  \
               .add_prefix('amenities_')

In [27]:
df_amenities.head()

amenities_ toilet  amenities_24hour checkin  \
listings_sf 0                  0                         1   
            1                  0                         0   
            2                  0                         0   
            3                  0                         0   
            5                  0                         0   

               amenities_Accessibleheight bed  \
listings_sf 0                               0   
            1                               0   
            2                               0   
            3                               0   
            5                               0   

               amenities_Accessibleheight toilet  amenities_Air conditioning  \
listings_sf 0                                  0                           0   
            1                                  0                           0   
            2                                  0                           0   
            3                                  0                           0   
            5                                  0                           0   

               amenities_Air purifier  amenities_Amazon Echo  \
listings_sf 0                       0                      0   
            1                       0                      0   
            2                       0                      0   
            3                       0                      0   
            5                       0                      0   

               amenities_BBQ grill  amenities_Baby bath  \
listings_sf 0                    0                    0   
            1                    0                    0   
            2                    0                    0   
            3                    0                    0   
            5                    1                    0   

               amenities_Baby monitor  \
listings_sf 0                       0   
            1                       0   
            2                       0   
            3                       0   
            5                       0   

                                      ...                          \
listings_sf 0                         ...                           
            1                         ...                           
            2                         ...                           
            3                         ...                           
            5                         ...                           

               amenities_Wide clearance to bed  \
listings_sf 0                                0   
            1                                0   
            2                                0   
            3                                0   
            5                                0   

               amenities_Wide clearance to shower  amenities_Wide doorway  \
listings_sf 0                                   0                       0   
            1                                   0                       0   
            2                                   0                       0   
            3                                   0                       0   
            5                                   0                       0   

               amenities_Wide entryway  amenities_Wide hallway clearance  \
listings_sf 0                        0                                 0   
            1                        0                                 0   
            2                        0                                 0   
            3                        0                                 0   
            5                        0                                 0   

               amenities_Wifi  amenities_Window guards  amenities_Wine cooler  \
listings_sf 0               1                        0                      0   
            1               1                        0                      0   


In [28]:
# There is a long list of amenities. This is to filter down to common amenities.

common_amenities=[]

for column in df_amenities.columns:
    value = df_amenities[column].quantile(0.75)
    if value>0:
        common_amenities.append(column)

common_amenities

['amenities_Air conditioning',
 'amenities_Bed linens',
 'amenities_Cable TV',
 'amenities_Carbon monoxide detector',
 'amenities_Coffee maker',
 'amenities_Cooking basics',
 'amenities_Dishes and silverware',
 'amenities_Dryer',
 'amenities_Essentials',
 'amenities_Extra pillows and blankets',
 'amenities_Familykid friendly',
 'amenities_Fire extinguisher',
 'amenities_First aid kit',
 'amenities_Free parking on premises',
 'amenities_Free street parking',
 'amenities_Hair dryer',
 'amenities_Hangers',
 'amenities_Heating',
 'amenities_Hot water',
 'amenities_Internet',
 'amenities_Iron',
 'amenities_Kitchen',
 'amenities_Laptop friendly workspace',
 'amenities_Lock on bedroom door',
 'amenities_Long term stays allowed',
 'amenities_Microwave',
 'amenities_Oven',
 'amenities_Private entrance',
 'amenities_Refrigerator',
 'amenities_Self checkin',
 'amenities_Shampoo',
 'amenities_Smoke detector',
 'amenities_Stove',
 'amenities_TV',
 'amenities_Washer',
 'amenities_Wifi']

In [29]:
df_common_amenities = df_amenities[common_amenities]
df_common_amenities.head()

amenities_Air conditioning  amenities_Bed linens  \
listings_sf 0                           0                     0   
            1                           0                     0   
            2                           0                     0   
            3                           0                     0   
            5                           0                     0   

               amenities_Cable TV  amenities_Carbon monoxide detector  \
listings_sf 0                   1                                   1   
            1                   0                                   0   
            2                   0                                   1   
            3                   0                                   1   
            5                   1                                   1   

               amenities_Coffee maker  amenities_Cooking basics  \
listings_sf 0                       0                         0   
            1                       0                         0   
            2                       0                         0   
            3                       0                         0   
            5                       1                         1   

               amenities_Dishes and silverware  amenities_Dryer  \
listings_sf 0                                0                1   
            1                                0                1   
            2                                0                1   
            3                                0                1   
            5                                1                1   

               amenities_Essentials  amenities_Extra pillows and blankets  \
listings_sf 0                     1                                     0   
            1                     1                                     0   
            2                     0                                     0   
            3                     0                                     0   
            5                     1                                     0   

                    ...        amenities_Oven  amenities_Private entrance  \
listings_sf 0       ...                     0                           1   
            1       ...                     0                           1   
            2       ...                     0                           1   
            3       ...                     0                           1   
            5       ...                     1                           0   

               amenities_Refrigerator  amenities_Self checkin  \
listings_sf 0                       0                       1   
            1                       0                       0   
            2                       0                       0   
            3                       0                       0   
            5                       1                       0   

               amenities_Shampoo  amenities_Smoke detector  amenities_Stove  \
listings_sf 0                  1                         1                0   
            1                  1                         1                0   
            2                  0                         1                0   
            3                  0                         1                0   
            5                  1                         1                1   

               amenities_TV  amenities_Washer  amenities_Wifi  
listings_sf 0             1                 1               1  
            1             0                 1               1  
            2             1                 1               1  
            3             1                 1               1  
            5             1                 1               1  

[5 rows x 36 columns]

In [30]:
df_listings_refined = pd.concat([df_smaller_listings, df_common_amenities], axis=1)

In [31]:
df_listings_refined.shape

(15379, 79)

In [32]:
df_listings_refined.drop('amenities', axis=1, inplace=True)

In [33]:
df_listings_refined.columns

Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'is_location_exact', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'minimum_nights', 'number_of_reviews',
       'first_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month', 'district',
       'amenities_Air conditioning', 

In [34]:
# This is to convert timestamp feature to duration

def duration(starting_time):
    """
    Calculate duration or number of years between current year and input years
    """
    starting_year = starting_time.year
    current_year = datetime.now().year
    duration = current_year - starting_year + 1
    
    return duration

In [35]:
# Calculate durations for timestamp columns

df_listings_refined['host_yrs'] = df_listings_refined['host_since'].apply(lambda x: duration(x))
df_listings_refined['yrs_since_first_review'] = df_listings_refined['first_review'].apply(lambda x: duration(x))

In [36]:
# Drop original and un-used timestamp features

time_stamps =['host_since', 'first_review']
df_listings_refined.drop(time_stamps, axis=1, inplace=True)

In [37]:
df_listings_refined.isnull().sum()

id                                         0
host_response_time                      1803
host_response_rate                      1803
host_is_superhost                          0
host_listings_count                        0
host_total_listings_count                  0
host_has_profile_pic                       0
host_identity_verified                     0
neighbourhood_cleansed                     0
city                                      14
is_location_exact                          0
property_type                              0
room_type                                  0
accommodates                               0
bathrooms                                 30
bedrooms                                   4
beds                                      15
bed_type                                   0
price                                      0
security_deposit                        3763
cleaning_fee                            2051
guests_included                            0
minimum_ni

In [38]:
# Filled null values with average for security deposits and cleaning fee

df_listings_refined['security_deposit'].fillna(
    value=df_listings_refined['security_deposit'].mean(),
    inplace=True)

df_listings_refined['cleaning_fee'].fillna(
    value=df_listings_refined['cleaning_fee'].mean(),
    inplace=True)

In [39]:
# Security deposit can vary in big range. Convert it to boolean varialbes

df_listings_refined['security_deposit'] = (df_listings_refined['security_deposit']>0).astype(int)

In [40]:
df_listings_refined.describe(include='all')

,id,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,city,...,amenities_Refrigerator,amenities_Self checkin,amenities_Shampoo,amenities_Smoke detector,amenities_Stove,amenities_TV,amenities_Washer,amenities_Wifi,host_yrs,yrs_since_first_review
count,1.537900e+04,13576,13576.000000,15379,15379.000000,15379.000000,15379,15379,15379,15365,...,15379.000000,15379.000000,15379.000000,15379.000000,15379.000000,15379.000000,15379.000000,15379.000000,15379.000000,12876.000000
unique,NaN,4,NaN,2,NaN,NaN,2,2,178,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,within an hour,NaN,f,NaN,NaN,t,t,San Jose,San Francisco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,8950,NaN,9888,NaN,NaN,15330,9031,1777,6168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.470061e+07,NaN,0.961138,NaN,25.504194,25.504194,NaN,NaN,NaN,NaN,...,0.370765,0.433123,0.825281,0.922037,0.292412,0.729046,0.726965,0.977177,4.873659,2.927695
std,8.398490e+06,NaN,0.124995,NaN,122.547116,122.547116,NaN,NaN,NaN,NaN,...,0.483025,0.495523,0.379739,0.268123,0.454885,0.444467,0.445533,0.149345,2.123994,1.733904
min,9.580000e+02,NaN,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,7.396749e+06,NaN,1.000000,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,2.000000
50%,1.575996e+07,NaN,1.000000,NaN,2.000000,2.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,5.000000,3.000000
75%,2.198158e+07,NaN,1.000000,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,4.000000


In [41]:
df_listings_refined.review_scores_rating.describe(include='all')

count    12764.000000
mean        95.435914
std          6.903849
min         20.000000
25%         94.000000
50%         97.000000
75%        100.000000
max        100.000000
Name: review_scores_rating, dtype: float64

In [42]:
df_listings_refined.review_scores_accuracy.describe(include='all')

count    12755.000000
mean         9.742140
std          0.690268
min          2.000000
25%         10.000000
50%         10.000000
75%         10.000000
max         10.000000
Name: review_scores_accuracy, dtype: float64

In [43]:
# Those varialbes have big range. Need to narrow down range to enable better modeling (take log10)


df_listings_refined['cleaning_fee'] = df_listings_refined['cleaning_fee'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_rating_log10'] = df_listings_refined['review_scores_rating'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_accuracy_log10'] = df_listings_refined['review_scores_accuracy'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_checkin_log10'] = df_listings_refined['review_scores_checkin'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_cleanliness_log10'] = df_listings_refined['review_scores_cleanliness'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_communication_log10'] = df_listings_refined['review_scores_communication'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_location_log10'] = df_listings_refined['review_scores_location'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_value_log10'] = df_listings_refined['review_scores_value'].apply(lambda x: np.log10(x) if x>0 else 0)


In [44]:
# Drop original ratings

review_scores =['review_scores_rating','review_scores_accuracy',
                'review_scores_checkin','review_scores_cleanliness', 
                'review_scores_communication', 'review_scores_location', 
                'review_scores_value']
df_listings_refined.drop(review_scores, axis=1, inplace=True)

In [45]:
# Minimum nights can vary in big range. Convert it to boolean varialbes

df_listings_refined['minimum_nights'] = (df_listings_refined['minimum_nights']>1).astype(int)
df_listings_refined.minimum_nights.head()

listings_sf  0    0
             1    1
             2    1
             3    1
             5    1
Name: minimum_nights, dtype: int64

In [46]:
df_listings_refined.isnull().sum()

id                                        0
host_response_time                     1803
host_response_rate                     1803
host_is_superhost                         0
host_listings_count                       0
host_total_listings_count                 0
host_has_profile_pic                      0
host_identity_verified                    0
neighbourhood_cleansed                    0
city                                     14
is_location_exact                         0
property_type                             0
room_type                                 0
accommodates                              0
bathrooms                                30
bedrooms                                  4
beds                                     15
bed_type                                  0
price                                     0
security_deposit                          0
cleaning_fee                              0
guests_included                           0
minimum_nights                  

In [47]:
# Drop rest of Null values

df_listings_cleaned = df_listings_refined.dropna()

In [48]:
df_listings_cleaned.shape

(10352, 78)

In [49]:
df_listings_cleaned.isnull().sum()

id                                     0
host_response_time                     0
host_response_rate                     0
host_is_superhost                      0
host_listings_count                    0
host_total_listings_count              0
host_has_profile_pic                   0
host_identity_verified                 0
neighbourhood_cleansed                 0
city                                   0
is_location_exact                      0
property_type                          0
room_type                              0
accommodates                           0
bathrooms                              0
bedrooms                               0
beds                                   0
bed_type                               0
price                                  0
security_deposit                       0
cleaning_fee                           0
guests_included                        0
minimum_nights                         0
number_of_reviews                      0
jurisdiction_nam

In [50]:
df_listings_cleaned.columns

Index(['id', 'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'is_location_exact', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'minimum_nights', 'number_of_reviews',
       'jurisdiction_names', 'instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'reviews_per_month', 'district', 'amenities_Air conditioning',
       'amenities_Bed linens', 'amenities_Cable TV',
       'amenities_Carbon monoxide detector', 'amenities_Coffee maker',
       'amenities_Cooking basics', 'amenities_Dishes and silverware',
       'amenities_Dryer', 'amenities_Essentials'

In [51]:
df_jurisdiction = df_listings_cleaned.loc[:, ['id', 'jurisdiction_names']]

In [52]:
# select non-numeric variables and create dummies

categorical_variables = df_listings_cleaned.select_dtypes(include=['object']).columns
df_listings_cleaned[categorical_variables].head()

host_response_time host_is_superhost host_has_profile_pic  \
listings_sf 0      within an hour                 t                    t   
            1      within an hour                 f                    t   
            2  within a few hours                 f                    t   
            3  within a few hours                 f                    t   
            5        within a day                 f                    t   

              host_identity_verified neighbourhood_cleansed           city  \
listings_sf 0                      t       Western Addition  San Francisco   
            1                      t         Bernal Heights  San Francisco   
            2                      t         Haight Ashbury  San Francisco   
            3                      t         Haight Ashbury  San Francisco   
            5                      f       Western Addition  San Francisco   

              is_location_exact property_type        room_type  bed_type  \
listings_sf 0                 t     Apartment  Entire home/apt  Real Bed   
            1                 t     Apartment  Entire home/apt  Real Bed   
            2                 t     Apartment     Private room  Real Bed   
            3                 t     Apartment     Private room  Real Bed   
            5                 t     Apartment  Entire home/apt  Real Bed   

              jurisdiction_names instant_bookable is_business_travel_ready  \
listings_sf 0      SAN FRANCISCO                f                        f   
            1      SAN FRANCISCO                f                        f   
            2      SAN FRANCISCO                f                        f   
            3      SAN FRANCISCO                f                        f   
            5      SAN FRANCISCO                f                        f   

                       cancellation_policy require_guest_profile_picture  \
listings_sf 0                     moderate                             f   
            1  strict_14_with_grace_period                             f   
            2  strict_14_with_grace_period                             f   
            3  strict_14_with_grace_period                             f   
            5                     moderate                             f   

              require_guest_phone_verification       district  
listings_sf 0                                f  San Francisco  
            1                                f  San Francisco  
            2                                f  San Francisco  
            3                                f  San Francisco  
            5                                f  San Francisco

In [53]:
categorical_variables.shape

(17,)

In [55]:
df_model = pd.get_dummies(data=df_listings_cleaned,columns=categorical_variables)

In [56]:
df_model.shape

(10352, 349)

In [57]:
df_model.columns

Index(['id', 'host_response_rate', 'host_listings_count',
       'host_total_listings_count', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'security_deposit',
       ...
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60',
       'require_guest_profile_picture_f', 'require_guest_profile_picture_t',
       'require_guest_phone_verification_f',
       'require_guest_phone_verification_t', 'district_Oakland',
       'district_San Francisco', 'district_Santa Clara',
       'district_Santa Cruz'],
      dtype='object', length=349)

In [58]:
# Create correlation matrix
corr_matrix = df_model.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

In [59]:
to_drop

['host_total_listings_count',
 'amenities_Refrigerator',
 'amenities_Stove',
 'amenities_Washer',
 'review_scores_checkin_log10',
 'review_scores_cleanliness_log10',
 'review_scores_communication_log10',
 'host_is_superhost_t',
 'host_has_profile_pic_t',
 'host_identity_verified_t',
 'city_Cupertino',
 'city_Palo Alto',
 'city_San Jose',
 'city_Santa Clara',
 'city_Sunnyvale',
 'is_location_exact_t',
 'room_type_Private room',
 'jurisdiction_names_Cupertino, CA',
 'jurisdiction_names_OAKLAND',
 'jurisdiction_names_PALO ALTO',
 'jurisdiction_names_SAN FRANCISCO',
 'jurisdiction_names_SAN JOSE',
 'jurisdiction_names_SANTA CLARA',
 'jurisdiction_names_Sunnyvale, CA',
 'instant_bookable_t',
 'require_guest_profile_picture_t',
 'require_guest_phone_verification_t',
 'district_Oakland',
 'district_San Francisco',
 'district_Santa Cruz']

In [60]:
# Drop features 
df_model.drop(to_drop, axis=1, inplace=True)

In [61]:
df_model.dropna(inplace=True)

In [62]:
df_model.columns

Index(['id', 'host_response_rate', 'host_listings_count', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'price', 'security_deposit',
       'cleaning_fee',
       ...
       'is_business_travel_ready_f', 'cancellation_policy_flexible',
       'cancellation_policy_moderate', 'cancellation_policy_strict',
       'cancellation_policy_strict_14_with_grace_period',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60',
       'require_guest_profile_picture_f', 'require_guest_phone_verification_f',
       'district_Santa Clara'],
      dtype='object', length=319)

In [63]:
# Store as csv file for feature investigation

csv_path = '/Users/xzhou/github/project_archives/files_airbnb/columns_check.csv'

df_model.to_csv(csv_path)

In [64]:
# Store the complete set of features

pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

pkl_listings = pkl_file.format('all_listings')

df_model.to_pickle(pkl_listings)

In [65]:
# Perform initial modelings to get a sense of performance, and for future selection

X = df_model.drop(['id','price'], axis=1)
y = df_model['price']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [67]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [73]:
model = sm.OLS(y_train, sm.add_constant(X_train_trans))
fit = model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     49.07
Date:                Mon, 17 Sep 2018   Prob (F-statistic):               0.00
Time:                        00:40:11   Log-Likelihood:                -39468.
No. Observations:                7246   AIC:                         7.953e+04
Df Residuals:                    6947   BIC:                         8.159e+04
Df Model:                         298                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        153.6855      0.674    228.156      0.000     152.365     155.006
x1             1.4151      1.181      1.198      0.231      -0.900       3.730
x2             3.4840      0.965      3.609      0.000       1.592       5.376
x3            24.3231      1.621     15.007      0.000      21.146      27.500
x4             5.2574      0.861      6.107      0.000       3.570       6.945
x5            26.9007      1.278     21.041      0.000      24.394      29.407
x6            -2.3378      1.390     -1.682      0.093      -5.063       0.387
x7            -1.2307      0.737     -1.669      0.095      -2.676       0.214
x8             4.6833      0.873      5.366      0.000       2.972       6.394
x9             4.7490      0.895      5.308      0.000       2.995       6.503
x10           -2.2201      0.816     -2.720      0.007      -3.820      -0.620
x11           -4.9341      1.220     -4.045      0.000      -7.325      -2.543
x12          -11.2432      1.016    -11.066      0.000     -13.235      -9.252
x13           -7.0002      1.174     -5.964      0.000      -9.301      -4.699
x14            3.3389      0.883      3.780      0.000       1.607       5.071
x15            0.2383      1.305      0.183      0.855      -2.321       2.797
x16            3.1271      0.823      3.801      0.000       1.514       4.740
x17            1.3474      0.851      1.583      0.113      -0.321       3.016
x18            3.6830      1.381      2.667      0.008       0.976       6.390
x19            0.3643      1.799      0.202      0.840      -3.162       3.891
x20           -0.0092      1.973     -0.005      0.996      -3.877       3.858
x21            1.6513      0.814      2.029      0.042       0.056       3.247
x22           -2.7810      0.787     -3.535      0.000      -4.323      -1.239
x23           -1.0505      1.195     -0.879      0.379      -3.393       1.292
x24            0.0953      0.777      0.123      0.902      -1.427       1.618
x25           -0.1204      0.813     -0.148      0.882      -1.715       1.474
x26            1.4421      0.791      1.823      0.068      -0.108       2.992
x27            1.5306      0.907      1.687      0.092      -0.248       3.309
x28            1.5301      0.905      1.692      0.091      -0.243       3.303
x29            0.5804      0.917      0.633      0.527      -1.217       2.377
x30            0.0543      0.911      0.060      0.952      -1.732       1.841
x31           -0.1897      0.735     -0.258      0.796      -1.630       1.251
x32            0.9529      0.972      0.980      0.327      -0.953       2.858
x33            1.9821      0.839      2.363      0.018       0.338       3.626
x34           -0.0897      0.923     -0.097      0.923      -1.900       1.720
x35           -0.5349      0.866     -0.618      0.537      -2.232       1.162
x3

In [74]:
X_train_trans

array([[ 0.29889922, -0.13925227, -0.09659807, ...,  0.2152993 ,
         0.2359082 , -0.60786131],
       [ 0.29889922, -0.13925227, -0.59126208, ...,  0.2152993 ,
         0.2359082 , -0.60786131],
       [ 0.29889922, -0.16103505, -0.09659807, ...,  0.2152993 ,
         0.2359082 ,  1.64511211],
       ...,
       [ 0.29889922, -0.12836088, -1.0859261 , ...,  0.2152993 ,
         0.2359082 , -0.60786131],
       [ 0.29889922, -0.13925227, -0.59126208, ...,  0.2152993 ,
         0.2359082 , -0.60786131],
       [ 0.29889922, -0.13925227, -0.59126208, ...,  0.2152993 ,
         0.2359082 ,  1.64511211]])

In [75]:
rf = RandomForestRegressor(n_estimators=500, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
rf.fit(X_train_trans, y_train)
y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 405.718, test: 2884.882
R^2 train: 0.959, test: 0.697


In [76]:
feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [77]:
feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)

In [78]:
feature_lists

[['bedrooms', 0.4395051810776777],
 ['cleaning_fee', 0.07548634741665643],
 ['room_type_Entire home/apt', 0.05210216196428506],
 ['reviews_per_month', 0.030616943326388334],
 ['city_San Francisco', 0.023855576753102584],
 ['city_Oakland', 0.021439991857963716],
 ['bathrooms', 0.020062816705203445],
 ['accommodates', 0.019963979294194033],
 ['number_of_reviews', 0.019683442343555495],
 ['host_listings_count', 0.016574790484920176],
 ['review_scores_rating_log10', 0.016220977738156356],
 ['host_yrs', 0.015688093642184035],
 ['guests_included', 0.012770873289267017],
 ['calculated_host_listings_count', 0.010171681964214344],
 ['review_scores_location_log10', 0.009198141142719852],
 ['beds', 0.008426475074116814],
 ['yrs_since_first_review', 0.007850830851113651],
 ['host_response_rate', 0.006379545300177394],
 ['amenities_Air conditioning', 0.0060425767787575845],
 ['review_scores_value_log10', 0.005128262103464437],
 ['jurisdiction_names_Santa Cruz County, CA', 0.004891423000971399],
 ['

In [79]:
# Keep features that have above 0.003 thresold

features_to_keep=[]

for (feature, importance) in feature_lists:
    if importance>=0.003:
        features_to_keep.append(feature)
    else:
        break

features_to_keep

['bedrooms',
 'cleaning_fee',
 'room_type_Entire home/apt',
 'reviews_per_month',
 'city_San Francisco',
 'city_Oakland',
 'bathrooms',
 'accommodates',
 'number_of_reviews',
 'host_listings_count',
 'review_scores_rating_log10',
 'host_yrs',
 'guests_included',
 'calculated_host_listings_count',
 'review_scores_location_log10',
 'beds',
 'yrs_since_first_review',
 'host_response_rate',
 'amenities_Air conditioning',
 'review_scores_value_log10',
 'jurisdiction_names_Santa Cruz County, CA',
 'amenities_Cable TV',
 'neighbourhood_cleansed_Palo Alto',
 'host_is_superhost_f',
 'amenities_Lock on bedroom door',
 'amenities_Private entrance',
 'amenities_Familykid friendly',
 'district_Santa Clara',
 'neighbourhood_cleansed_San Jose',
 'host_identity_verified_f']

In [80]:
len(features_to_keep)

30

In [81]:
no_features_kept = len(features_to_keep)
features_w_importance = feature_lists[0: no_features_kept]
features_w_importance

[['bedrooms', 0.4395051810776777],
 ['cleaning_fee', 0.07548634741665643],
 ['room_type_Entire home/apt', 0.05210216196428506],
 ['reviews_per_month', 0.030616943326388334],
 ['city_San Francisco', 0.023855576753102584],
 ['city_Oakland', 0.021439991857963716],
 ['bathrooms', 0.020062816705203445],
 ['accommodates', 0.019963979294194033],
 ['number_of_reviews', 0.019683442343555495],
 ['host_listings_count', 0.016574790484920176],
 ['review_scores_rating_log10', 0.016220977738156356],
 ['host_yrs', 0.015688093642184035],
 ['guests_included', 0.012770873289267017],
 ['calculated_host_listings_count', 0.010171681964214344],
 ['review_scores_location_log10', 0.009198141142719852],
 ['beds', 0.008426475074116814],
 ['yrs_since_first_review', 0.007850830851113651],
 ['host_response_rate', 0.006379545300177394],
 ['amenities_Air conditioning', 0.0060425767787575845],
 ['review_scores_value_log10', 0.005128262103464437],
 ['jurisdiction_names_Santa Cruz County, CA', 0.004891423000971399],
 ['

In [82]:
# Perform another model(Random Forest) based on selected features

X_updated = df_model[features_to_keep]
y_updated = df_model['price']

In [83]:
X_updated_train, X_updated_test, y_updated_train, y_updated_test = train_test_split(
    X_updated, y_updated, test_size =0.3, random_state=42)

In [84]:
ssX2 = StandardScaler()
X_updated_train_trans = ssX2.fit_transform(X_updated_train)
X_updated_test_trans = ssX2.transform (X_updated_test)

In [85]:
rf_updated = RandomForestRegressor(n_estimators=500, 
                                   criterion='mse', 
                                   random_state=3, 
                                   n_jobs=-1)
rf_updated.fit(X_updated_train_trans, y_updated_train)
y_updated_train_pred = rf_updated.predict(X_updated_train_trans)
y_updated_test_pred = rf_updated.predict(X_updated_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_updated_train, y_updated_train_pred),
        mean_squared_error(y_updated_test, y_updated_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_updated_train, y_updated_train_pred),
        r2_score(y_updated_test, y_updated_test_pred)))

MSE train: 415.273, test: 3053.745
R^2 train: 0.958, test: 0.679


In [109]:
columns =['id']
columns.extend(features_to_keep)

In [110]:
df_listings_top_features = df_model[columns]
df_listings_top_features.head()

id  bedrooms  cleaning_fee  room_type_Entire home/apt  \
listings_sf 0   958       1.0       2.00000                          1   
            1  5858       2.0       2.00000                          1   
            2  7918       1.0       1.69897                          0   
            3  8142       1.0       1.69897                          0   
            5  8567       2.0       2.09691                          1   

               reviews_per_month  city_San Francisco  city_Oakland  bathrooms  \
listings_sf 0               1.38                   1             0        1.0   
            1               0.99                   1             0        1.0   
            2               0.16                   1             0        4.0   
            3               0.15                   1             0        4.0   
            5               0.27                   1             0        1.0   

               accommodates  number_of_reviews            ...             \
listings_sf 0             3                152            ...              
            1             5                112            ...              
            2             2                 17            ...              
            3             2                  7            ...              
            5             6                 30            ...              

               jurisdiction_names_Santa Cruz County, CA  amenities_Cable TV  \
listings_sf 0                                         0                   1   
            1                                         0                   0   
            2                                         0                   0   
            3                                         0                   0   
            5                                         0                   1   

               neighbourhood_cleansed_Palo Alto  host_is_superhost_f  \
listings_sf 0                                 0                    0   
            1                                 0                    1   
            2                                 0                    1   
            3                                 0                    1   
            5                                 0                    1   

               amenities_Lock on bedroom door  amenities_Private entrance  \
listings_sf 0                               0                           1   
            1                               0                           1   
            2                               1                           1   
            3                               1                           1   
            5                               0                           0   

               amenities_Familykid friendly  district_Santa Clara  \
listings_sf 0                             1                     0   
            1                             1                     0   
            2                             0                     0   
            3                             1                     0   
            5                             1                     0   

               neighbourhood_cleansed_San Jose  host_identity_verified_f  
listings_sf 0                                0                         0  
            1                                0                         0  
            2                                0                         0  
            3                                0                         0  
            5                                0                         1  

[5 rows x 31 columns]

In [111]:
# Add District informaiton for future use

df_listings_top_features = pd.merge(left=df_listings_top_features, 
                                    right=df_district, 
                                    how ='left', 
                                    left_on='id',
                                    right_on='id')

df_listings_top_features.head()

,id,bedrooms,cleaning_fee,room_type_Entire home/apt,reviews_per_month,city_San Francisco,city_Oakland,bathrooms,accommodates,number_of_reviews,...,amenities_Cable TV,neighbourhood_cleansed_Palo Alto,host_is_superhost_f,amenities_Lock on bedroom door,amenities_Private entrance,amenities_Familykid friendly,district_Santa Clara,neighbourhood_cleansed_San Jose,host_identity_verified_f,district
0,958,1.0,2.00000,1,1.38,1,0,1.0,3,152,...,1,0,0,0,1,1,0,0,0,San Francisco
1,5858,2.0,2.00000,1,0.99,1,0,1.0,5,112,...,0,0,1,0,1,1,0,0,0,San Francisco
2,7918,1.0,1.69897,0,0.16,1,0,4.0,2,17,...,0,0,1,1,1,0,0,0,0,San Francisco
3,8142,1.0,1.69897,0,0.15,1,0,4.0,2,7,...,0,0,1,1,1,1,0,0,0,San Francisco
4,8567,2.0,2.09691,1,0.27,1,0,1.0,6,30,...,1,0,1,0,0,1,0,0,1,San Francisco


In [112]:
# Add Jurisdition informaiton for future use

df_listings_top_features = pd.merge(left=df_listings_top_features, 
                                    right=df_jurisdiction, 
                                    how ='left', 
                                    left_on='id',
                                    right_on='id')

df_listings_top_features.head()

,id,bedrooms,cleaning_fee,room_type_Entire home/apt,reviews_per_month,city_San Francisco,city_Oakland,bathrooms,accommodates,number_of_reviews,...,neighbourhood_cleansed_Palo Alto,host_is_superhost_f,amenities_Lock on bedroom door,amenities_Private entrance,amenities_Familykid friendly,district_Santa Clara,neighbourhood_cleansed_San Jose,host_identity_verified_f,district,jurisdiction_names
0,958,1.0,2.00000,1,1.38,1,0,1.0,3,152,...,0,0,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
1,5858,2.0,2.00000,1,0.99,1,0,1.0,5,112,...,0,1,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
2,7918,1.0,1.69897,0,0.16,1,0,4.0,2,17,...,0,1,1,1,0,0,0,0,San Francisco,SAN FRANCISCO
3,8142,1.0,1.69897,0,0.15,1,0,4.0,2,7,...,0,1,1,1,1,0,0,0,San Francisco,SAN FRANCISCO
4,8567,2.0,2.09691,1,0.27,1,0,1.0,6,30,...,0,1,0,0,1,0,0,1,San Francisco,SAN FRANCISCO


In [113]:
# Check for null values before storing

df_listings_top_features.isnull().sum()

id                                          0
bedrooms                                    0
cleaning_fee                                0
room_type_Entire home/apt                   0
reviews_per_month                           0
city_San Francisco                          0
city_Oakland                                0
bathrooms                                   0
accommodates                                0
number_of_reviews                           0
host_listings_count                         0
review_scores_rating_log10                  0
host_yrs                                    0
guests_included                             0
calculated_host_listings_count              0
review_scores_location_log10                0
beds                                        0
yrs_since_first_review                      0
host_response_rate                          0
amenities_Air conditioning                  0
review_scores_value_log10                   0
jurisdiction_names_Santa Cruz Coun

In [114]:
# Rename columns used for furture reference to avoid confusion

df_listings_top_features.rename(columns={'district':'ref_district',
                                        'jurisdiction_names': 'ref_jurisdiction'},
                               inplace=True)
df_listings_top_features

,id,bedrooms,cleaning_fee,room_type_Entire home/apt,reviews_per_month,city_San Francisco,city_Oakland,bathrooms,accommodates,number_of_reviews,...,neighbourhood_cleansed_Palo Alto,host_is_superhost_f,amenities_Lock on bedroom door,amenities_Private entrance,amenities_Familykid friendly,district_Santa Clara,neighbourhood_cleansed_San Jose,host_identity_verified_f,ref_district,ref_jurisdiction
0,958,1.0,2.000000,1,1.38,1,0,1.0,3,152,...,0,0,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
1,5858,2.0,2.000000,1,0.99,1,0,1.0,5,112,...,0,1,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
2,7918,1.0,1.698970,0,0.16,1,0,4.0,2,17,...,0,1,1,1,0,0,0,0,San Francisco,SAN FRANCISCO
3,8142,1.0,1.698970,0,0.15,1,0,4.0,2,7,...,0,1,1,1,1,0,0,0,San Francisco,SAN FRANCISCO
4,8567,2.0,2.096910,1,0.27,1,0,1.0,6,30,...,0,1,0,0,1,0,0,1,San Francisco,SAN FRANCISCO
5,8739,1.0,1.698970,0,5.41,1,0,1.0,3,594,...,0,0,0,0,1,0,0,1,San Francisco,SAN FRANCISCO
6,9225,1.0,1.698970,0,3.84,1,0,1.0,2,411,...,0,1,0,0,0,0,0,0,San Francisco,SAN FRANCISCO
7,10251,2.0,2.000000,1,2.69,1,0,1.0,6,291,...,0,0,0,1,1,0,0,0,San Francisco,SAN FRANCISCO
8,10578,0.0,1.875061,1,0.22,1,0,1.0,2,18,...,0,1,0,0,0,0,0,0,San Francisco,SAN FRANCISCO
9,10819,3.0,2.318063,1,0.34,1,0,2.0,4,22,...,0,1,0,1,1,0,0,0,San Francisco,SAN FRANCISCO


In [116]:
# Store processed data with selected features for future use

pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'

pkl_path = pkl_file.format('listings_with_selected_features')

df_listings_top_features.to_pickle(pkl_path)